In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


train=pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
y=train.pop('Cover_Type')
id_=train.pop('Id')
id_test=test.pop('Id')

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier( task_type = 'GPU',devices = '0')
model.fit(train, y)



In [ ]:
feature_importances=pd.DataFrame(model.get_feature_importance(),columns=['importance'])

In [ ]:
feature=pd.DataFrame(train.columns,columns=['feature'])

In [ ]:
o=pd.concat([feature_importances,feature],axis=1)

In [ ]:
o

In [ ]:

plt.figure(figsize=(16, 16))
sns.barplot(data=o.sort_values(by='importance', ascending=False), x='importance', y='feature')
plt.title('Feature Importances ')  


In [ ]:
o=o[o['importance']>=0.1]['feature'].values

In [ ]:
train[o].head(2)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier( task_type = 'GPU',devices = '0')
model.fit(train[o], y)



In [ ]:
from sklearn.model_selection import cross_val_score
a_score = cross_val_score(model, train[o], y, cv=5)


In [ ]:
print('DecisionTreeClassifier Cross Validation Score', round(a_score.mean() * 100, 2).astype(str) + '%')

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
sub['Cover_Type'] =model.predict(test[o])
sub.to_csv("submission.csv",index=False)
sub.head()